In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as func
from scipy.io import loadmat
import numpy as np
from sklearn.feature_extraction import image
import pandas as pd
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import gensim

In [2]:
import re
import pickle
import json
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import math
from difflib import SequenceMatcher
import pandas as pd

In [3]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
year_regex = re.compile(r'((19[0-9]{2})|(20[0-9]{2}))[a-z]?')
conversion_dict = {}
stop_words = [',', '.', '(', ')', ':', '-', "+", ";", "a", "about", "al", "al.", "all", 
	"already", "also", "although", "am", "an", "and", "another", "any", "anyhow", "are", 
	"aren", "aren't", "around", "as", "at", "back", "be", "because", "been", 
	"being", "beyond", "but", "by", "can", "cannot", "cant", "co", "con", "could", "couldn", 
	"couldnt", "d", "de", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", 
	"doing", "don", "don't", "done", "due", "each", "either", "else", "elsewhere", "et", 
	"etc", "even", "ever", "except", "for", "found", "from", "further", "had", "hadn", 
	"hadn't", "has", "hasn", "hasn't", "hasnt", "have", "haven", "haven't", "having", 
	"he", "hence", "her", "here", "hereafter", "hereby", "hers", 
	"herself", "him", "himself", "his", "how", "however", "i", "ie", "if", "in", "inc", 
	"indeed", "interest", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", 
	"just", "ltd", "ll", "m", "may", "me", "meanwhile", "might", "mightn", 
	"mightn't", "mine", "moreover", "most", "mostly", "move", "much", "must", "mustn", 
	"mustn't", "my", "myself", "name", "namely", "need", "needn", "needn't", "neither", 
	"nevertheless", "no", "nobody", "noone", "nor", "not", "now", "nowhere", "o", "of", 
	"off", "often", "on", "only", "onto", "or", "other", "others", "otherwise", "our", "own", 
	"per", "perhaps", "put", "rather", "re", "s", "same", "see", "seem", "seemed", 
	"seeming", "seems", "serious", "she", "should", "shouldn", "shouldn't", "since", 
	"sincere", "so", "some", "somehow", "someone", "something", "somewhere", "still", 
	"such", "t", "take", "than", "that", "that'll", "the", "their", "theirs", 
	"them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", 
	"therein", "thereupon", "these", "they", "this", "those", "though", "throughout", 
	"thru", "thus", "to", "together", "too", "toward", "towards", "un", "until", "upon", 
	"us", "ve", "very", "via", "was", "wasn", "wasn't", "we", "well", "were", "weren", 
	"weren't", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", 
	"whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", 
	"whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", 
	"without", "won", "won't", "would", "wouldn", "wouldn't", "y", "yet", "you", "your", 
	"yours", "yourself", "yourselves", "from SVM import SVCone", "two", "three", "four", "five", "six", "seven",
	"eight", "nine", "zero", "between", 'below', 'ourselves', "you'll", 'again', 'once', 'over', 'shan', 'few', 
    'against', 'before', 'out', 'down', 'both', 'up', "you've", "shan't", "you're", "should've", 'ours', 'ma', 
    "couldn't", 'during', 'more', 'ain', 'through', 'after', 'above', "she's", "you'd", 'under' ]

In [4]:
def get_citations(folder):
    citation_list = {}
    for file in os.listdir(folder) :
        tree = ET.parse(folder+file)
        root = tree.getroot()
        id = file[:8]
        for element in root.iterfind("algorithm"):
            if(element.attrib['name']=="ParsCit"):
                citlist = element.getchildren()
                cits = citlist[0].getchildren()
                citations = []
                for cit in cits:
                    cit_dict = {}
                    if(cit.attrib['valid']=="true"):
                        try :
                            title = cit.find('title').text.lower()
                        except :
                            title = cit.find('rawString').text.lower()
                        cit_dict['title'] = title
                        cit_dict['cit'] = cit
                        citations.append(cit_dict)
                
                citation_list[id] = citations
            
    return citation_list 

In [5]:
citations = get_citations("../xmls/")

In [6]:
def get_words(string, cit_auths=None):

    string = string.replace('-', '').lower()
    context_words = word_tokenize(string)
    final_context_words = []

    for word in context_words:
        if word in stop_words:
            continue
        if re.fullmatch(year_regex, word): #or match_auths(word, cit_auths):
            continue
        if re.fullmatch(r'[0-9]+([.][0-9]+)?', word):
            word = '<number>'
            final_context_words.append(word)
        elif 'this_citation' in word:
            final_context_words.append('<this_citation>')
        elif re.fullmatch(r'[a-z]+', word):
            original_word = word
            final_word = lemmatizer.lemmatize(word)
            
            if final_word not in conversion_dict:
                conversion_dict[final_word] = {}

            if original_word not in conversion_dict[final_word]:
                conversion_dict[final_word][original_word] = 0
            conversion_dict[final_word][original_word]+=1

            final_context_words.append(final_word)

    return final_context_words

In [7]:
def get_contexts(citations) :
    dataset = {}
    count = 0
    for key in citations.keys():
        context_list = []
        for cit in citations[key] :
            dict1 = {}
            dict1['paper_name'] = cit['title']
            dict1['context'] = ""
            for context in cit['cit'].findall('contexts/context') :
                text = context.text.lower()
                citstr = context.get('citStr').lower()
                text = text.replace(citstr, "this_citation")
                dict1['context']+= text+" "
            dict1['context'] = get_words(dict1['context'])
            context_list.append(dict1)
        dataset[key] = context_list
        count+=1
        if(count%100==0) :
            print(count)
    return dataset

In [8]:
contexts = get_contexts(citations)

100
200
300
400
500
600
700
800
900
1000
1100


In [9]:
max1 = 0
avg = 0
number = 0
for key in contexts :
    papers = contexts[key]
    for pap in papers :
        val = len(pap['context'])
        avg += val
        number += 1
        if(val>max1) :
            max1 = val
            
print(max1)
print(avg/number)

1141
140.41749366058292


In [10]:
embeddings_dict = {}
with open("glove.6B.100d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [11]:
print(len(embeddings_dict.keys()))

400000


In [12]:
mapping = {}
mapping_rev = {}
values = []
count = 0
for key in contexts :
    papers = contexts[key]
    for pap in papers :
        name = pap['paper_name']
        words = pap['context']
        if(len(words)>0) :
            mapping[key+"_|_"+name] = count
            mapping_rev[count] = key+"_|_"+name
            count+=1
            embed = []
            for word in words :
                if(word in embeddings_dict) :
                    embed.append(embeddings_dict[word])
                else :
                    embed.append(embeddings_dict['unk'])
            diff = 1141-len(words)
            if(diff>0) :
                for i in range(diff) :
                    embed.append(embeddings_dict['unk'])
            values.append(embed)  

In [13]:
print(len(values))

27635


In [14]:
tags = pickle.load(open("../pickles_data/baseline_tags.pkl","rb"))

In [15]:
output = []
for i in range(count) :
    key,name = mapping_rev[i].split("_|_")
    papers = tags[key]
    for pap in papers :
        if(pap['paper_name']==name) :
            output.append(pap['tag'])
            break

In [16]:
print(len(output))

27635


In [17]:
location_feature = pickle.load(open("../pickles_data/location_feature.pkl","rb"))
title_overlap = pickle.load(open("../pickles_data/title_overlap.pkl","rb"))
popularity = pickle.load(open("../pickles_data/popularity_sent.pkl", "rb"))
paper_info = pickle.load(open("../pickles/paper_info (1).pickle", "rb"))
year_diff = pickle.load(open("../pickles_data/year_diff.pkl","rb"))
num_table = pickle.load(open("../pickles_data/num_table.pkl","rb"))
author_feat = pickle.load(open("../pickles/author_level.pickle", "rb"))

In [18]:
extra = []
for i in range(count) :
    extra_val = []
    key,name = mapping_rev[i].split("_|_")
    for pap in location_feature[key] :
        if(pap['paper_name']==name) :
            extra_val.extend(pap['location_feature'])
            break
            
    for pap in title_overlap[key] :
        if(pap['paper_name']==name) :
            extra_val.append(pap['overlap'])
            break
            
    for pap in popularity[key] :
        if(pap['paper_name']==name) :
            extra_val.append(pap['popularity'])
            break
            
            
    for pap in num_table[key] :
        if(pap['paper_name']==name) :
            extra_val.append(pap['num_table'])
            break
            
#     for pap in author_feat[key] :
#         if(pap['paper_name']==name) :
#             extra_val.append(pap['hindex'])
#             extra_val.append(pap['productivity'])
#             break
            
    for pap in year_diff[key] :
        if(pap['paper_name']==name) :
            extra_val.append(pap['diff'])
            break
            
#     for pap in paper_info[key] :
#         if(pap['paper_name']==name) :
#             try: 
#                 extra_val.append( int(pap['citation']) )
#             except :
#                 extra_val.append(1)
#             try :
#                 extra_val.append(int(pap['venue_cit']) )
#             except :
#                 extra_val.append(20)
                
#             break
                
            
    extra.append(extra_val)

In [19]:
print(len(extra[0]))
c=0
for i in range(1,len(extra)) :
    if(len(extra[i])!=10):
        c+=1
        print(len(extra[i]))
        
#         print(i)
#         print(len(extra[i]))
#         print(len(extra[i-1]))
#         print(extra[i])
#         print(extra[i-1])
#         break

print(c)

10
0


In [32]:
def shuffle(train_data, extra, train_output) :
    baselines = []
    non_baselines = []
    base_extra = []
    non_base_extra = []
    for i in range(len(train_output)) :
        if(train_output[i]==1) :
            ar = []
            ar.append(train_data[i])
            ar.append(extra[i])
            baselines.append(ar)
#             base_extra.append(extra[i])
        else :
            ar = []
            ar.append(train_data[i])
            ar.append(extra[i])
            non_baselines.append(ar)
    
    n = len(baselines)
    ar = np.random.choice(len(non_baselines), len(baselines))
    nb_ar = []
    nb_extra = []
    for x in ar :
        nb_ar.append(non_baselines[x])
        
    data = []
    data.extend(nb_ar)
    output = []
    for i in range(len(data)):
        output.append(0)
    
    data.extend(baselines)
    for i in range(n) :
        output.append(1)
        
    print(data[0])
    total_data = []
    for i in range(len(data)):
        ar = []
        ar.append(data[i])
        ar.append(output[i])
        total_data.append(ar)
        
    total_data = np.array(total_data)
    np.random.shuffle(total_data)
    
    data = []
    extra = []
    output = []
    for ar in total_data :
        data.append(list(ar[0][0]))
        extra.append(list(ar[0][1]))
        output.append(ar[1])
        
    data = np.array(data)
    extra = np.array(extra)
    output = np.array(output)
    
    return data, extra, output

In [33]:
values1, extra1, output1 = shuffle(values, extra, output)

[[array([-1.0889e+00,  1.5505e-01,  3.1952e-01,  2.8231e-01, -2.6882e-01,
        1.4778e-01,  3.5143e-01,  3.6910e-01, -1.4636e-01,  4.6264e-01,
        7.2031e-01, -9.3214e-01, -2.5624e-01,  5.9452e-01,  3.6191e-01,
       -1.7785e-01, -3.8944e-01,  4.6279e-01,  2.5420e-02,  4.9203e-02,
        1.0677e+00, -1.4154e-04,  2.0156e-01, -4.1351e-01, -5.0160e-02,
        5.1361e-01,  1.1236e+00,  3.1335e-01,  6.2001e-01, -3.1925e-01,
        8.5093e-01,  7.5885e-01, -2.4657e-01,  1.8882e-01,  5.1676e-01,
        2.2059e-01,  1.0004e+00, -7.4258e-02,  5.9846e-01, -2.5869e-01,
        2.8310e-01, -9.1471e-01, -3.3909e-01, -6.4210e-01, -1.9543e-01,
        6.3651e-02, -8.5830e-01,  1.2382e-01, -1.2064e-01,  2.8015e-01,
        5.3761e-02, -3.4761e-01,  5.7299e-01,  1.9325e-01, -9.2623e-01,
       -1.2383e+00,  1.3549e-01,  4.6227e-01,  9.6163e-01,  3.9299e-01,
       -7.0814e-01, -5.8321e-01, -8.9628e-01, -3.8967e-01,  1.4761e+00,
        1.0776e+00,  7.6046e-01,  3.0343e-01,  6.0218e-01,  4.

In [34]:
# print(np.array(extra).shape)
# print(np.array(values).shape)
# print(np.array(output).shape)

print(values1.shape)
print(extra1.shape)
print(output1.shape)

(5474, 1141, 100)
(5474, 10)
(5474,)


In [35]:
print(extra1[0])

[ 0.   0.   6.   6.   6.   0.   0.3 17.   1.   7. ]


In [36]:
len(extra[0])

10

In [37]:
size = len(values1)
trainset = values1[:int(0.8*size)]
trainlabel = output1[:int(0.8*size)]
testset = values1[int(0.8*size):]
testlabel = output1[int(0.8*size):]
extratrain = extra1[:int(0.8*size)] 
extratest = extra1[int(0.8*size):]

In [38]:
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.kaiming_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim 
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b

#         print("eij",eij.shape)
#         print("x",x.contiguous().view(-1,feature_dim).shape)
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / (torch.sum(a, 1, keepdim=True) + 1e-10)

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

class Attention_Net(nn.Module):
    def __init__(self):
        super(Attention_Net, self).__init__()
        drp = 0.1
        max_features = 1141
        embed_size = 100
        # self.embedding = nn.Embedding(max_features, embed_size)
        # # self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        # self.embedding.weight.requires_grad = False

        # self.embedding_dropout = nn.Dropout2d(0.1)
        self.lstm = nn.LSTM(embed_size, 32, bidirectional=True, batch_first=True)
#         self.lstm2 = nn.LSTM(64*2, 32, bidirectional=True, batch_first=True)

        self.attention_layer = Attention(64, 1141)
        
        self.linear = nn.Linear(74 , 32)
        self.relu = nn.ReLU()
        self.out = nn.Linear(32, 2)

    def forward(self, x, extra):
        h_embedding = x
        h_lstm, _ = self.lstm(h_embedding)
#         h_lstm, _ = self.lstm2(h_lstm)
        h_lstm_atten = self.attention_layer(h_lstm)
#         print(h_lstm_atten.shape)
#         print(extra.shape)
        next_inp = torch.cat((h_lstm_atten,extra),1)
       
#         print(next_inp.shape)
        conc = self.relu(self.linear(next_inp))
        out = self.out(conc)
        return out

In [39]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 64

train_data = TensorDataset(torch.as_tensor(trainset),  torch.as_tensor(extratrain), torch.as_tensor(trainlabel))
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(torch.as_tensor(testset), torch.as_tensor(extratest),torch.as_tensor(testlabel))
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=64)

In [40]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

def eval_model(model, val_loader) :
    final_out = []
    final_lab = []

    for idx, (val_input, extra, val_label) in enumerate(val_loader):
    # val_input = val_input.permute(1,0,2)
    # print(train_input.shape)
        val_input = val_input.type(torch.FloatTensor)
        val_input = val_input.cuda()
        extra = extra.type(torch.FloatTensor)
        extra = extra.cuda()
        output = model(val_input,extra)
        output = output.cpu().detach().numpy()
        val_label = val_label.cpu().detach().numpy()
        ar= []
        for i in range(output.shape[0]) :
            index = -1
            max_val = -1
            for j in range(len(output[i])) :
                if(output[i][j]>max_val) :
                    index = j
                    max_val = output[i][j]
            ar.append(index)
    
        val_label = list(val_label)
        final_out.extend(ar)
        final_lab.extend(val_label)

        
    return classification_report(final_lab, final_out)

In [41]:
learning_rate = 0.005
num_epochs = 25
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model = Attention_Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for batch_id, (train_input,extra, train_label) in enumerate(train_dataloader):
        optimizer.zero_grad() 
        train_input = train_input.type(torch.FloatTensor)
        train_input = train_input.cuda()
        extra = extra.type(torch.FloatTensor)
        extra = extra.cuda()
        output = model(train_input,extra)

        train_label = train_label.type(torch.LongTensor)
        train_label = train_label.cuda()
        loss = criterion(output, train_label)
        loss.backward()
        optimizer.step()
        
        if batch_id % 100 == 0:
            print('Epoch [{}/{}], Loss:{:.4f}'
                .format(epoch+1, num_epochs, loss.data))
            print("classification_report")
            print(eval_model(model,val_dataloader))
            print("--------------------------------------------------------------")
            # losses.append(loss.data)
            
        del train_input
        del train_label
        del output
        del loss
        torch.cuda.empty_cache()

Epoch [1/25], Loss:0.7458
classification_report
              precision    recall  f1-score   support

           0       0.64      0.82      0.72       533
           1       0.77      0.57      0.65       562

    accuracy                           0.69      1095
   macro avg       0.70      0.69      0.68      1095
weighted avg       0.71      0.69      0.68      1095

--------------------------------------------------------------
Epoch [2/25], Loss:0.4138
classification_report
              precision    recall  f1-score   support

           0       0.75      0.85      0.80       533
           1       0.84      0.73      0.78       562

    accuracy                           0.79      1095
   macro avg       0.79      0.79      0.79      1095
weighted avg       0.79      0.79      0.79      1095

--------------------------------------------------------------
Epoch [3/25], Loss:0.3899
classification_report
              precision    recall  f1-score   support

           0       0.

Epoch [20/25], Loss:0.4351
classification_report
              precision    recall  f1-score   support

           0       0.85      0.77      0.81       533
           1       0.80      0.87      0.83       562

    accuracy                           0.82      1095
   macro avg       0.82      0.82      0.82      1095
weighted avg       0.82      0.82      0.82      1095

--------------------------------------------------------------
Epoch [21/25], Loss:0.3489
classification_report
              precision    recall  f1-score   support

           0       0.83      0.81      0.82       533
           1       0.83      0.84      0.83       562

    accuracy                           0.83      1095
   macro avg       0.83      0.83      0.83      1095
weighted avg       0.83      0.83      0.83      1095

--------------------------------------------------------------
Epoch [22/25], Loss:0.2884
classification_report
              precision    recall  f1-score   support

           0      

In [42]:
print(eval_model(model, val_dataloader))

              precision    recall  f1-score   support

           0       0.82      0.85      0.83       533
           1       0.85      0.82      0.83       562

    accuracy                           0.83      1095
   macro avg       0.83      0.83      0.83      1095
weighted avg       0.83      0.83      0.83      1095



In [25]:
torch.save(model.state_dict, "models/1layer_bilstm_atten_feat_statedict.pt")

/home/tanisha17116/.local/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Attention_Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/tanisha17116/.local/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [26]:
torch.save(model, "models/1layer_bilstm_atten_feat_model.pt")

In [ ]:
class Attention_Net_Features(nn.Module) :
    
    